## Simulation Framework 

In [33]:
import import_ipynb
import math 
import importlib
import random
import copy

# import functions from model_evaluation.ipynb  
import model_evaluation 

In [18]:
# simulation parameters 

# number of groups
n_groups = 4 

# number of agents 
n_agents = 4

### Group Distributions 

In [19]:
# create random distributions 
def distribute_agents(n_agents, n_groups):
    """
    This function randomly selects a subset of group indices for each agent. The number of groups for each agent is randomly chosen between 1 and the total number of groups.
    :returns (list of lists): Each inner list represents the agents assigned to a particular group.
    """
    group_dist = [[] for _ in range(n_groups)]

    for agent in range(n_agents):
        groups = random.sample(range(n_groups), random.randint(1, n_groups))
        for group in groups:
            group_dist[group].append(agent)

    return group_dist

In [35]:
# Create random group distributions with a gradual transition from one to the other extreme 
def gradual_distribution_transition(n_agents, n_groups, iterations):
    """
    This function starts with a distribution where each group contains only one agent and gradually modifies it in each iteration.
    :returns (list of lists): Each inner list represents the agents assigned to a particular group.
    """
    # Initial distribution: Each group contains only one agent
    group_dist = [[agent] for agent in range(n_agents)]

    # List to store distributions at each iteration
    distributions = [copy.deepcopy(group_dist)]

    for _ in range(iterations):  # Start from iteration 1
        # Randomly select a group
        source_group = random.randint(0, n_groups - 1)
        
        # Randomly select a destination group different from the source group
        possible_destination_groups = [group for group in range(n_groups) if group != source_group]
        destination_group = random.choice(possible_destination_groups)

        # Randomly select an agent from the source group
        if group_dist[source_group]:
            agent_to_move = random.choice(group_dist[source_group])

            # Move the agent to the destination group if not already present
            if agent_to_move not in group_dist[destination_group]:
                group_dist[destination_group].append(agent_to_move)
            else:
                continue

        # Save the current distribution (deep copy to avoid referencing the same object)
        distributions.append(copy.deepcopy(group_dist))

    return distributions

In [37]:
# Example usage
n_agents = 4
n_groups = 4
iterations = 100  # Adjust the number of iterations as needed
result = gradual_distribution_transition(n_agents, n_groups, iterations)
print(result)
len(result)

[[[0], [1], [2], [3]], [[0], [1], [2, 1], [3]], [[0], [1], [2, 1], [3, 0]], [[0], [1, 3], [2, 1], [3, 0]], [[0, 3], [1, 3], [2, 1], [3, 0]], [[0, 3], [1, 3, 0], [2, 1], [3, 0]], [[0, 3], [1, 3, 0], [2, 1], [3, 0, 1]], [[0, 3], [1, 3, 0], [2, 1, 3], [3, 0, 1]], [[0, 3, 2], [1, 3, 0], [2, 1, 3], [3, 0, 1]], [[0, 3, 2, 1], [1, 3, 0], [2, 1, 3], [3, 0, 1]], [[0, 3, 2, 1], [1, 3, 0], [2, 1, 3, 0], [3, 0, 1]], [[0, 3, 2, 1], [1, 3, 0], [2, 1, 3, 0], [3, 0, 1, 2]], [[0, 3, 2, 1], [1, 3, 0, 2], [2, 1, 3, 0], [3, 0, 1, 2]]]


13

### Vote Distribution 

In [23]:
# define fixed vote distribution 
def vote_distribution_fixed(n_agents, votes_agent):
    return [votes_agent] * n_agents

In [24]:
vote_dist = vote_distribution_fixed(n_agents=n_agents, votes_agent=5)
print(vote_dist)

[5, 5, 5, 5]


In [25]:
# Define input variables  
group_dist = [[0], [1], [2], [3]] 
vote_dist = [5, 5, 5, 5]

In [26]:
# Assuming this is the correct function signature
result_group_memberships = model_evaluation.group_memberships(group_dist, vote_dist)
print(result_group_memberships)

[[0], [1], [2], [3]]


In [27]:
result_groups, result_aggregate_weight = model_evaluation.first_term(group_dist, vote_dist, result_group_memberships)
print(result_groups)
print(result_aggregate_weight)

{'group_0': {'agent_0': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_1': {'agent_1': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_2': {'agent_2': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_3': {'agent_3': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}}
20.0


In [28]:
result_individual, result_aggregated = model_evaluation.interaction_terms(group_dist, result_group_memberships, vote_dist)
print(result_individual)
print(result_aggregated)

{'group_0': {'other_group_1': {'components_int_term1': {'agent_i_0': {'vote_attenuation_i': 5, 'num_groups_i': 1, 'vote_weight_i': 5.0}}, 'components_int_term2': {'agent_j_1': {'vote_attenuation_j': 5, 'num_groups_j': 1, 'vote_weight_j': 5.0}}, 'sum_vote_weight_i': 5.0, 'sqrt_sum_vote_weight_i': 2.23606797749979, 'sum_vote_weight_j': 5.0, 'sqrt_sum_vote_weight_j': 2.23606797749979, 'multiplied_interaction': 5.000000000000001}, 'other_group_2': {'components_int_term1': {'agent_i_0': {'vote_attenuation_i': 5, 'num_groups_i': 1, 'vote_weight_i': 5.0}}, 'components_int_term2': {'agent_j_2': {'vote_attenuation_j': 5, 'num_groups_j': 1, 'vote_weight_j': 5.0}}, 'sum_vote_weight_i': 5.0, 'sqrt_sum_vote_weight_i': 2.23606797749979, 'sum_vote_weight_j': 5.0, 'sqrt_sum_vote_weight_j': 2.23606797749979, 'multiplied_interaction': 5.000000000000001}, 'other_group_3': {'components_int_term1': {'agent_i_0': {'vote_attenuation_i': 5, 'num_groups_i': 1, 'vote_weight_i': 5.0}}, 'components_int_term2': {'

In [29]:
plurality_score = model_evaluation.connection_oriented_cluster_match(result_aggregate_weight, result_aggregated)
print(plurality_score)

8.94427190999916


In [30]:
result_quadratic_voting, result_sum_quadratic_votes = model_evaluation.quadratic_voting(vote_dist)
print(result_quadratic_voting)
print(result_sum_quadratic_votes)

{0: 2.23606797749979, 1: 2.23606797749979, 2: 2.23606797749979, 3: 2.23606797749979}
8.94427190999916


In [31]:
# Jaccard Index 
pairwise_jaccard_matrix = model_evaluation.pairwise_jaccard_similarity(group_dist)
print("Pairwise Jaccard Similarity Matrix:")
for row in pairwise_jaccard_matrix:
    print(row)

global_similarity = model_evaluation.global_jaccard_similarity(pairwise_jaccard_matrix)
print("Global Jaccard Similarity:", global_similarity)

Pairwise Jaccard Similarity Matrix:
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
Global Jaccard Similarity: 0.0
